In [12]:
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import MaximumLikelihoodEstimator, HillClimbSearch
from itertools import combinations 
import itertools

In [13]:
df = pd.read_csv('test_after_agg.csv',sep=';')

In [14]:
df_training = df.drop(columns = ['CO Number','Host Country','Data Year','Allocation month'])

In [15]:
df_training = df_training.iloc[0:1000,]

In [16]:
demo_columns = list(df_training.columns[0:10])

In [17]:
demo_columns.extend(df_training.columns[360:397])

In [18]:
wc_columns = list(df_training.columns[110:360])

In [19]:
combinations_list = list(itertools.product(demo_columns, wc_columns))

In [20]:
len(combinations_list)

11500

In [21]:
def encode_wc(columns_list):
    counter = 1
    dict_encodings = {}
    for code in columns_list:
        dict_encodings[code] = counter
        counter+=1
    encoded_columns = columns_list.map(lambda x: dict_encodings[x])
    encoding_mappings = {v: k for k, v in dict_encodings.items()}
    return encoded_columns,encoding_mappings
    

In [22]:
needed_combinations = []
for comb in combinations_list:
    if not df_training[(df_training[comb[0]] == 1) & (df_training[comb[1]] == 1)].empty:
        needed_combinations.append(comb)

In [23]:
len(needed_combinations)

1042

In [29]:
model = BayesianModel(needed_combinations)
model.fit(data=df_training, estimator=HillClimbSearch(data=df_training))
cpds = model.get_cpds()

TypeError: issubclass() arg 1 must be a class

In [14]:
encoded_columns, encoding_mappings = encode_wc(df_training.columns)
df_training.columns=encoded_columns

In [28]:
for cpd in cpds:
    print(cpd.values)
    print(cpd.variable)
    print(cpd.cardinality)
    print(cpd)

[1.]
Dual Career Partner Name
[1]
+-------------------------------+---+
| Dual Career Partner Name(1.0) | 1 |
+-------------------------------+---+
[0.124 0.876]
Family members at host
[2]
+-----------------------------+-------+
| Family members at host(0.0) | 0.124 |
+-----------------------------+-------+
| Family members at host(1.0) | 0.876 |
+-----------------------------+-------+
[1.]
Married
[1]
+--------------+---+
| Married(1.0) | 1 |
+--------------+---+
[0.809 0.191]
f
[2]
+--------+-------+
| f(0.0) | 0.809 |
+--------+-------+
| f(1.0) | 0.191 |
+--------+-------+
[1.]
germany
[1]
+--------------+---+
| germany(1.0) | 1 |
+--------------+---+
[0.962 0.038]
level 1
[2]
+--------------+-------+
| level 1(0.0) | 0.962 |
+--------------+-------+
| level 1(1.0) | 0.038 |
+--------------+-------+
[0.746 0.254]
level 2
[2]
+--------------+-------+
| level 2(0.0) | 0.746 |
+--------------+-------+
| level 2(1.0) | 0.254 |
+--------------+-------+
[0.458 0.542]
level 3
[2]
+-------

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [89]:
model.get_immoralities()

{('Dual Career Partner Name', 'Family members at host'),
 ('Dual Career Partner Name', 'Married'),
 ('Dual Career Partner Name', 'f'),
 ('Dual Career Partner Name', 'germany'),
 ('Dual Career Partner Name', 'level 1'),
 ('Dual Career Partner Name', 'level 2'),
 ('Dual Career Partner Name', 'level 3'),
 ('Dual Career Partner Name', 'level 4'),
 ('Dual Career Partner Name', 'm'),
 ('Dual Career Partner Name', 'no management level'),
 ('Family members at host', 'Married'),
 ('Family members at host', 'f'),
 ('Family members at host', 'germany'),
 ('Family members at host', 'level 1'),
 ('Family members at host', 'level 2'),
 ('Family members at host', 'level 3'),
 ('Family members at host', 'level 4'),
 ('Family members at host', 'm'),
 ('Family members at host', 'no management level'),
 ('Married', 'f'),
 ('Married', 'germany'),
 ('Married', 'level 1'),
 ('Married', 'level 2'),
 ('Married', 'level 3'),
 ('Married', 'level 4'),
 ('Married', 'm'),
 ('Married', 'no management level'),
 ('f'

In [19]:
model.get_independencies()

KeyboardInterrupt: 

In [29]:
print(model)
import networkx as nx
import pylab as matplotlib
nx.draw_networkx(model, with_labels=False)

/home/camelboy/daniel/env/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:522: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not cb.is_string_like(edge_color) \
/home/camelboy/daniel/env/lib/python3.6/site-packages/networkx/drawing/nx_pylab.py:543: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if cb.is_string_like(edge_color) or len(edge_color) == 1:


In [34]:
nx.drawing.nx_agraph.write_dot(model,'graph-example.dot')

In [35]:
import pydot

(graph,) = pydot.graph_from_dot_file('graph-example.dot')
graph.write_png('graph-example.png')